In [ ]:
from quickdb.sqlhttp import sqlclient

In [ ]:
sql = '''
SELECT count(*) from pdr2_wide GROUP BY object_id % 10
'''

%time res = sqlclient.post_sql_with_tqdm(sql)
res.dataframe()

In [ ]:
sql = '''
SELECT count(*) from pdr2_wide GROUP BY forced.isprimary
'''

%time res = sqlclient.post_sql_with_tqdm(sql)
res.dataframe()

In [ ]:
sql = '''
SELECT
    histogram(flux2mag(forced.i.psfflux_flux), range => (10, 30), bins => 50)
FROM
    pdr2_dud
'''

%time res = sqlclient.post_sql_with_tqdm(sql)

from matplotlib import pyplot

hist, bins = res.target_list[1][0]
pyplot.plot(bins[1:], hist)
pyplot.grid()

In [ ]:
sql = '''
SELECT
    histogram2d(
        flux2mag(forced.g.psfflux_flux) - flux2mag(forced.r.psfflux_flux),
        flux2mag(forced.r.psfflux_flux) - flux2mag(forced.i.psfflux_flux),   
        bins => (200, 400), range => ((-0.5, 2.), (-0.5, 2.5)))
FROM
    pdr2_wide
WHERE
    forced.i.extendedness_value < 0.5
'''

%time res = sqlclient.post_sql_with_tqdm(sql)

import numpy
hist, xedges, yedges = res.target_list[1][0]
pyplot.imshow(numpy.log(1 + hist).T, origin='low', extent=(xedges[0], xedges[-1], yedges[0], yedges[-1]))